In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from matplotlib.ticker import ScalarFormatter
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
import os

if os.name == 'posix':
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="Malgun Gothic")

In [3]:
df = pd.read_csv('C:/Users/Home/Desktop/Data/2차/CardData_new.csv', encoding = 'cp949')

In [5]:
d = {
    '21.Active_연속' : 0,
    '22.Active_간헐' : 1,
    '34.휴면_9M+' : 2,
    '32.휴면_6M-' : 2,
    '33.휴면_9M-' : 2,
    '31.휴면_3M' : 2
}
df['고객구분'] = df['고객구분'].map(d)

In [6]:
df2 = df.copy()

In [7]:
df2['이용건수_신용_B0M'].value_counts()

0      62116
1      38308
2      15646
3      10565
4       8461
       ...  
245        1
318        1
355        1
261        1
260        1
Name: 이용건수_신용_B0M, Length: 271, dtype: int64

In [15]:
df2['이용건수_체크_B0M'].value_counts()

 0      259808
 1        5588
 2        3168
 3        2187
 4        1744
         ...  
 282         1
 168         1
 158         1
 302         1
-28          1
Name: 이용건수_체크_B0M, Length: 191, dtype: int64

In [20]:
df2['Recency'] = (df2['이용건수_신용_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_체크_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_카드론_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_카드론_R3M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['RP건수_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_온라인_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_오프라인_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_페이_오프라인_R3M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_페이_온라인_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_페이_오프라인_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용개월수_전체_R3M'].apply(lambda x: 0 if x <= 0 else 1))

In [22]:
df2['Recency'].value_counts().sort_index()

0     37844
1     19894
2      6154
3     49885
4     80288
5     37126
6     41396
7     13836
8     11192
9      2250
10      113
11       22
Name: Recency, dtype: int64

In [26]:
(df2['_1순위카드이용건수'] <= df2['이용건수_신용_R12M']).value_counts()   

True     299933
False        67
dtype: int64

In [29]:
(df2['_1순위카드이용건수'] <= df2['이용건수_신용_R12M'] + df2['이용건수_체크_R12M']).value_counts()

True     299937
False        63
dtype: int64

In [30]:
df2['Frequency'] = df2['이용건수_신용_R12M']/12 + df2['이용건수_체크_R12M']/12 + df2['이용건수_카드론_R12M']/12

In [35]:
(df2['이용금액_업종기준'] <= df2['이용금액_신용_R12M']+ df2['이용금액_체크_R12M']+ df2['이용금액_카드론_R12M']).value_counts()

True     299060
False       940
dtype: int64

In [36]:
df2['Monetary'] = df2['이용금액_신용_R12M'] + df2['이용금액_체크_R12M'] + df2['이용금액_카드론_R12M'] + df2['이용금액_업종기준']

In [38]:
df2['이용금액대'].value_counts()

01.100만원+    61467
05.10만원-     59192
02.50만원+     54216
09.미사용       48480
03.30만원+     39667
04.10만원+     36978
Name: 이용금액대, dtype: int64

In [40]:
df2['이용개월수_신용_R12M'].value_counts().sort_index()

0      24932
1       7199
2       5993
3       5253
4       5416
5       5466
6       6392
7       9289
8       9054
9      10046
10     12919
11     18685
12    179356
Name: 이용개월수_신용_R12M, dtype: int64

In [44]:
df2['Monetary'].quantile(0.75)

14481927.5

In [43]:
df2['Recency'].value_counts().sort_index()

0     37844
1     19894
2      6154
3     49885
4     80288
5     37126
6     41396
7     13836
8     11192
9      2250
10      113
11       22
Name: Recency, dtype: int64

In [48]:
df2['Monetary'].nlargest(3000).iloc[-1]

67185525

실제 하나카드 VVIP 기준 소비금액 상위 3000명

In [49]:
df2['Monetary'] == 67185525

0         False
1         False
2         False
3         False
4         False
          ...  
299995    False
299996    False
299997    False
299998    False
299999    False
Name: Monetary, Length: 300000, dtype: bool

퍼스트 - 종합금액 7천 초과 // 벌써 어긋남

로열 - 종합금액 3300만원 초과

클래식 - 종합금액 1700만원 초과

자체기준  0 -> 0.2,

VIP

In [57]:
df2['Monetary'].quantile(0.5)

6375778.0

In [56]:
df2['Frequency'].quantile(0.2)

1.3333333333333333

In [ ]:
df2['Recency'] > 9 , 
df2['Frequency'] > 55.25, #(0.9)
df2['Monetary'] > 26192024 #(0.9)

In [58]:
df['탈회여부'].value_counts()

0    178046
1    121954
Name: 탈회여부, dtype: int64

충성고객

In [ ]:
df2['Recency'] > 9, 
df2['Frequency'] > 11 #(0.5),
df2['Monetary'] < 670380 #(0.2)

잠재 VIP 고객

In [ ]:
df2['Recency'] > 9,
df2['Frequency'] < 1.3, #(0.2)
df2['Monetary'] > 6375778 #(0.5)

잠재 충성 고객

In [ ]:
df2['Recency'] > 9,
df2['Frequency'] < 1.3, #(0.2)
df2['Monetary'] < 670380 #(0.2)

놓치면 안될 고객

In [ ]:
0 < df2['Recency'] <= 3,
df2['Frequency'] > 11 #(0.5),
df2['Monetary'] > 6375778 #(0.5)

관심필요고객

In [ ]:
0 < df2['Recency'] <= 3,
df2['Frequency'] > 11 #(0.5),
df2['Monetary'] < 670380 #(0.2)

이탈우려고객

In [ ]:
0 < df2['Recency'] <= 3,
df2['Frequency'] < 1.3, #(0.2)
df2['Monetary'] > 6375778 #(0.5)

겨울잠고객

In [ ]:
df2['Recency'] <= 3
df2['Frequency'] < 1.3, #(0.2)
df2['Monetary'] < 670380 #(0.2)